In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import zipline
from zipline.api import (
    order_target_percent,
    symbol,
    schedule_function,
    date_rules,
    time_rules,
)
from datetime import datetime
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt, ticker, rc

<IPython.core.display.Javascript object>

In [3]:
def initialize(context):
    # Securities and target weights
    context.securities = {
        "SPY": 0.25,
        "TLT": 0.3,
        "IEF": 0.3,
        "GLD": 0.075,
        "DBC": 0.075,
    }

    # Schedule rebalance for once a month
    schedule_function(rebalance, date_rules.month_start(), time_rules.market_open())

<IPython.core.display.Javascript object>

In [4]:
def rebalance(context, data):
    # Loop through the securities
    for sec, weight in context.securities.items():
        sym = symbol(sec)
        # Check if we can trade
        if data.can_trade(sym):
            # Reset the weight
            order_target_percent(sym, weight)

<IPython.core.display.Javascript object>

In [5]:
start = pd.Timestamp("1997-01-01 08:15:12", tz="UTC")
end = pd.Timestamp("2018-12-31 08:15:12", tz="UTC")

<IPython.core.display.Javascript object>

In [6]:
%%time
result = zipline.run_algorithm(
    start=start,  # Set start
    end=end,  # Set end
    initialize=initialize,  # Define startup function
    capital_base=100000,  # Set initial capital
    data_frequency="daily",  # Set data frequency
    bundle="ac_equities_db", # Select bundle i.e bring your own data!
)

UnknownBundle: No bundle registered with the name 'ac_equities_db'

<IPython.core.display.Javascript object>

Analyze the results:

In [7]:
# Format for book image
font = {"family": "eurostile", "weight": "normal", "size": 16}
rc("font", **font)

<IPython.core.display.Javascript object>

In [8]:
# Settings
window = 252
year_length = 252

<IPython.core.display.Javascript object>

In [9]:
# Copy the columsn we need
df = result.copy().filter(items=["portfolio_value", "gross_leverage"])

NameError: name 'result' is not defined

<IPython.core.display.Javascript object>

In [ ]:
# Function for annualized return
def ann_ret(ts):
    return np.power((ts[-1] / ts[0]), (year_length / len(ts))) - 1


# Function for drawdown
def dd(ts):
    return np.min(ts / np.maximum.accumulate(ts)) - 1

In [ ]:
# Get a rolling window
rolling_window = result.portfolio_value.rolling(window)

In [ ]:
# Calculate rolling analytics
df["annualized"] = rolling_window.apply(ann_ret)
df["drawdown"] = rolling_window.apply(dd)

In [ ]:
# Drop initial n/a values
df.dropna(inplace=True)

In [ ]:
# Make a figure
fig = plt.figure(figsize=(12, 12))

In [ ]:
# Make the base lower, just to make the graph easier to read
df["portfolio_value"] /= 100

In [ ]:
# First chart
ax = fig.add_subplot(411)
ax.set_title("Strategy Results")
ax.plot(
    df["portfolio_value"],
    linestyle="-",
    color="black",
    label="Equity Curve",
    linewidth=3.0,
)

# Set log scale
ax.set_yscale("log")

# Make the axis look nicer
ax.yaxis.set_ticks(
    np.arange(df["portfolio_value"].min(), df["portfolio_value"].max(), 500)
)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("%0.0f"))

# Add legend and grid
ax.legend()
ax.grid(False)

In [ ]:
# Second chart
ax = fig.add_subplot(412)
ax.plot(df['gross_leverage'], 
        label='Strategy exposure'.format(window), 
        linestyle='-', 
        color='black',
        linewidth=1.0)

# Make the axis look nicer
ax.yaxis.set_ticks(np.arange(df['gross_leverage'].min(), df['gross_leverage'].max(), 0.2 ))
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))

# Add legend and grid
ax.legend()
ax.grid(True)


In [ ]:
# Third chart
ax = fig.add_subplot(413)
ax.plot(df['annualized'], 
        label='{} days annualized return'.format(window), 
        linestyle='-', 
        color='black',
        linewidth=1.0)
ax.axhline(df['annualized'].mean(), color='black', linewidth=1.5, label='Mean {:0.1f}%'.format(df['annualized'].mean() * 100))

# Make the axis look nicer
ax.yaxis.set_ticks(np.arange(df['annualized'].min(), df['annualized'].max(), 0.1 ))
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.2f'))

# Add legend and grid
ax.legend()
ax.grid(True)


In [ ]:
# Fourth chart
ax = fig.add_subplot(414)
ax.plot(
    df["drawdown"],
    label="{} days max drawdown".format(window),
    linestyle="-",
    color="black",
    linewidth=1.0,
)

# Make the axis look nicer
ax.yaxis.set_ticks(np.arange(df["drawdown"].min(), df["drawdown"].max(), 0.02))
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("%0.2f"))

# Add legend and grid
ax.legend()
ax.grid(True)

In [ ]:
fig